In [104]:
import pandas as pd


In [105]:
#function to read excel_file 
def read_file(file_name):
    file = pd.ExcelFile(file_name)
    list = []
    
    #reading data in each sheet of the file and storing in a list
    for sheet_name in file.sheet_names:
        df = pd.read_excel(file, sheet_name = sheet_name)
        #averaging the values to the minute granularity 
        df['Date'] = pd.to_datetime(df['Date (UTC)']).dt.floor('min')
        df = df.drop(columns = ['Date (UTC)'])
        average_min = df.groupby('Date').mean()
        average_min = average_min.reset_index()
        average_min['Date'] = pd.to_datetime(average_min['Date'])
        list.append(average_min)
    return list 

#function to merge all the sheets' dataframe into one dataframe
def merge(list):
    #create a dataframe from the list of dataframes
    result = list[0]
    
    #merging all the dataframes based on the common date
    for index, data_frame in enumerate (list):
        if index < len(list) - 1:
            result = result.merge(list[index + 1], on = 'Date', how = 'inner')
    
    #changing columns' name
    new_cols = {}
    for column in result.columns:
        new_column = column.replace(' for PRODSQL\\LOCAL','').replace('SQL Server: ','').replace(' ','_').lower()
        new_cols[column] = new_column
    new_cols['Disk avg. read time for prodsql1-vm.wwwoodproducts.com > M: (SQL Data 1)'] = 'disk_avg_read_time_1'
    new_cols['Disk avg. read time for prodsql1-vm.wwwoodproducts.com > N: (SQL Data 2)'] = 'disk_avg_read_time_2'
    new_cols['Disk avg. read time for prodsql1-vm.wwwoodproducts.com > I: (SQL Index)'] = 'index1'
    new_cols['Disk avg. write time for prodsql1-vm.wwwoodproducts.com > M: (SQL Data 1)'] = 'disk_avg_write_time_1'
    new_cols['Disk avg. write time for prodsql1-vm.wwwoodproducts.com > N: (SQL Data 2)'] = 'disk_avg_write_time_2'
    new_cols['Disk avg. write time for prodsql1-vm.wwwoodproducts.com > I: (SQL Index)'] = 'index2'
    result.rename(columns = new_cols, inplace = True )
    
    return result

#function to handling data
def data_process(data_frame):
    #converting the Date (UTC) to Date(CST)
    from datetime import datetime
    import pytz
    utc_timezone = pytz.utc
    cst_timezone = pytz.timezone('America/Chicago')
    data_frame['date'] = data_frame['date'].dt.tz_localize('UTC').dt.tz_convert(cst_timezone)
    
    #converting Free Memory metric from Bytes to GB
    data_frame['free_memory'] = data_frame['free_memory']/1024**3
    
    #Multiplying the compilations/batch by 100
    data_frame['compilations/batch'] = data_frame['compilations/batch']*100
    
    #Combine Disk avg read time sql data 1 and Disk avg read time sql data 2 into one column Disk avg read time sql data using a SUM
    data_frame['disk_avg_read_time_sql_data']  = data_frame['disk_avg_read_time_1']+data_frame['disk_avg_read_time_2']
    return data_frame


In [106]:
#reading files, concatenating and writing to csv
def reading(file_name):
    list_of_file = read_file(file_name)
    df = merge(list_of_file)
    df = data_process(df)
    return df

datasetA1 = reading('1st_day_A.xlsx')
datasetA2 = reading('2nd_day_A.xlsx')
datasetA3 = reading('3rd_day_A.xlsx')
datasetA4 = reading('4th_day_A.xlsx')
datasetA5 = reading('5th_day_A.xlsx')

datasetA = pd.concat([datasetA1,datasetA2,datasetA3,datasetA4,datasetA5])
datasetA.to_csv('A.csv')

datasetB1 = reading('1st_day_B.xlsx')
datasetB2 = reading('2nd_day_B.xlsx')
datasetB3 = reading('3rd_day_B.xlsx')
datasetB4 = reading('4th_day_B.xlsx')
datasetB5 = reading('5th_day_B.xlsx')

datasetA = pd.concat([datasetB1,datasetB2,datasetB3,datasetB4,datasetB5],ignore_index= True)
datasetA.to_csv('B.csv')

In [107]:
#Reading data from csv file and filtering results
dataA = pd.read_csv('A.csv')
dataB = pd.read_csv('B.csv')

dataA['date'] = pd.to_datetime(dataA['date'])
dataA_filtered = dataA[(dataA['date'].dt.hour>=5) & (dataA['date'].dt.hour<= 16)]
dataA_filtered.to_csv("[Daytime]A.csv", index=False)

dataB['date'] = pd.to_datetime(dataB['date'])
dataB_filtered = dataB[(dataB['date'].dt.hour>=5) & (dataB['date'].dt.hour<= 16)]
dataB_filtered.to_csv("[Daytime]B.csv", index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7185 entries, 0 to 7184
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype                    
---  ------                       --------------  -----                    
 0   Unnamed: 0                   7185 non-null   int64                    
 1   date                         7185 non-null   datetime64[ns, UTC-05:00]
 2   batch_requests/sec           7185 non-null   float64                  
 3   user_connections             7185 non-null   float64                  
 4   processor_time               7185 non-null   float64                  
 5   free_memory                  7185 non-null   float64                  
 6   page_reads/sec               7185 non-null   float64                  
 7   disk_avg_read_time_1         7185 non-null   float64                  
 8   disk_avg_read_time_2         7185 non-null   float64                  
 9   index1                       7185 non-null   float64